In [1]:
import tweepy as tw
import pandas as pd
import time
import unicodecsv
from tweepy import OAuthHandler
import json
import pickle
import glob
import os

# Authenticate the Tweeter ID

Keep your information about tweeter API ID in the file named key.dat. Is the json formatted file

In [2]:
# reads json file key.dat to get the information about API ID
with open("key.dat",'r') as key:
    key_source = key.read().replace('\n','')
key_dict = json.loads(str(key_source))

In [3]:
# initializes the tweeter api informations
consumer_key = key_dict["consumer_key"]
consumer_secret = key_dict["consumer_secret"]
access_token = key_dict["access_token"]
access_secret = key_dict["access_secret"] 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
# Decorator to print out the error message
import pickle
def safe_run(func):
    def func_wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(e)
   
    return func_wrapper



# function to save the friend list in a list using pickle. The extension for the file is *.ntwk

@safe_run
def getFriendsSaved(node,path,file_no):
    outFileName = path+str(file_no)+".ntwk"
    outFile = open(outFileName, 'wb')
    ids_followers = []
    for page in tw.Cursor(api.followers_ids, id = node).pages():
        ids_followers.extend(page)
        time.sleep(15)
    ids_friends = []
    for page in tw.Cursor(api.friends_ids, id = node).pages():
        ids_friends.extend(page)
        time.sleep(15)
    two_way_friends = set(ids_followers).intersection(set(ids_friends))
    two_way_friends_list = [node] + list(two_way_friends)
    pickle.dump(two_way_friends_list, outFile)
    outFile.close()

    
# function to open the friend list saved in *.ntwk file format
@safe_run    
def openFriendList(path,file_no):
    inFileName = path+str(file_no)+".ntwk"
    inFile = open(inFileName, 'rb')
    friend_list = pickle.load(inFile)
    inFile.close()
    return friend_list[0],friend_list[1:]

In [ ]:
#maximum number of nodes that you want in your network
max_nodes = 100000  

# username for a node in tweeter around which you are trying to create network
# Here the default user_name is tweeter
parent_node_username = "tweeter"

user = api.get_user(screen_name = parent_node_username)
parent_node = user.id
# level of branches of friendship you are going to search
max_branch = 6

#output folder 
path = "data/"

# for the first branch
getFriendsSaved(parent_node, path, 1)
node = parent_node
nodeCount = 1
nodeIndexInBranch = []

friend_parent, friend_list = openFriendList(path,1 )

for friends in friend_list:
    nodeCount = nodeCount + 1
    getFriendsSaved(friends, path, nodeCount)
    print("%d file saved in %d branch."%(nodeCount, 1))
nodeInBranch.append(nodeCount)

nodeIndexInBranch.append(nodeCount)

for count in range(5,max_branch + 1):
    for branch in range(nodeIndexInBranch[count-2],nodeIndexInBranch[count-1]):
        try:
            friend_parent, friend_list = openFriendList(path, branch)
            for friends in friend_list:
                nodeCount = nodeCount + 1
                try:
                    print("%d file saved in %d branch."%(nodeCount, count))
                    getFriendsSaved(friends, path, nodeCount)
                except:
                    continue
        except:
            continue
            

    nodesInBranch.append(nodeCount)
    
    if(nodeCount>=max_nodes):
        print("Maximum nodes reached")
        sys.exit()
        
        
        